In [ ]:
#on charge le dataset et on le met dans un dataframe
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None
from folium.plugins import HeatMap
import folium
df = pd.read_csv('Data.csv', sep=',', low_memory=False)

In [ ]:
# On ne peut pas garder toutes les colonnes, un choix s'opère donc. 

# Nous scindons les colonnes restantes dans plusieurs dataframes :

# Data, dataframe général, possédant "eventid","iyear","imonth","iday","extended","resolution","country_txt","region_txt","provstate","city","success", "suicide","attacktype1_txt", "targtype1_txt","targsubtype1_txt","natlty1_txt","gname","nperps","claimed","weaptype1_txt","weapsubtype1_txt","nkill","nwound","dbsource"
Data = df[["eventid","iyear","imonth","iday","extended","resolution","country_txt","region_txt","provstate","city","success", "suicide","attacktype1_txt", "targtype1_txt","targsubtype1_txt","natlty1_txt","gname","nperps","claimed","weaptype1_txt","weapsubtype1_txt","nkill","nwound","dbsource"]]

#On remplace les valeurs vides de la colonne claimed par 0
Data['claimed'].fillna(0,inplace=True)
#On remplace les valeurs vides de la colonne nperps par "Inconnu"
Data['nperps'].fillna(-99.0,inplace=True)
#On remplace les valeurs vides de la colonne nkill par "Inconnu"
Data['nkill'].fillna(-99.0,inplace=True)
#On remplace les valeurs vides de la colonne nwound par "Inconnu"
Data['nwound'].fillna(-99.0,inplace=True)
#on remplace les valeurs vides de la colonne resolution par la date de l'attentat dans le format "jour/mois/année"
Data['resolution'].fillna(Data['iday'].astype(str) + "/" + Data['imonth'].astype(str) + "/" + Data['iyear'].astype(str),inplace=True)
#On drop les coordonnées null
Data = Data.dropna(subset=['latitude', 'longitude'])

Data.to_csv("Trait2.csv")

Trait = pd.read_csv('Trait2.csv', sep=',', low_memory=False)

In [ ]:
# geo, dataframe ayant pour but la géolocalisation, comprenant "eventid", "latitude", "longitude","region_txt","country_txt"
geo = df[["eventid", "latitude", "longitude","region_txt","country_txt"]].dropna()

geo.to_csv("Geo.csv")

In [ ]:
#headlines, dataframe comprenant les descriptions. "eventid","summary"
headlines = df[["eventid","summary"]]

In [ ]:
# On regarde le nombre de d'attentats revendiqués par rapport au nombre total 

Revendiqué = df[["claimed"]]

#On récupère le nombre de ligne ou la colonne "claimed" est égale à 1 dans la variable "count"

count = Revendiqué[Revendiqué.claimed == 1].count()

#On récupère le nombre de ligne total dans la variable "count2"

count2 = Revendiqué.count()

#On calcule le pourcentage de ligne ou la colonne "claimed" est égale à 1

pourcentage = (count/count2)*100

print(pourcentage)


In [ ]:
 # On regarde le nombre d'attentats par année pour voir comment évolue le terrorisme au fil des années

#On récupère le nombre d'attentats par année dans la variable "YearCount"

YearCount = df[["eventid","iyear"]].groupby('iyear').count()

In [ ]:
import folium
from folium.plugins import MarkerCluster, FeatureGroupSubGroup

# créer la carte centrée sur la position moyenne des événements
map = folium.Map(location=[geo['latitude'].mean(), geo['longitude'].mean()], zoom_start=4)

# créer un groupe de marqueurs clusterisés
marker_cluster = MarkerCluster().add_to(map)

# ajouter un marqueur pour chaque événement dans le groupe de marqueurs clusterisés
for index, row in geo.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']]).add_to(marker_cluster)

# créer un groupe pour afficher le nombre de marqueurs dans chaque cluster
sub_group = folium.plugins.FeatureGroupSubGroup(marker_cluster, 'Events Count')

# ajouter le groupe à la carte
map.add_child(sub_group)

# ajouter un compteur pour chaque cluster
for i, layer in enumerate(marker_cluster._children.values()):
    count = len(layer._children)
    bounds = layer.get_bounds()
    pos = [(bounds[0][0] + bounds[1][0]) / 2, (bounds[0][1] + bounds[1][1]) / 2]
    folium.Marker(location=pos, icon=folium.Icon(icon='info-sign'), tooltip=str(count)).add_to(sub_group)

map.save("CarteGenerale.html")

In [ ]:
def create_heatmap_for_region(region_data, region_name):
    # Créer une carte centrée sur les coordonnées moyennes de la région
    map_center = [region_data['latitude'].mean(), region_data['longitude'].mean()]
    region_map = folium.Map(location=map_center, zoom_start=4)

    # Préparez les données pour la heatmap
    heat_data = [[row['latitude'], row['longitude']] for index, row in region_data.iterrows()]

    # Ajoutez la heatmap à la carte
    HeatMap(heat_data).add_to(region_map)

    # Sauvegarder la heatmap dans un fichier HTML
    region_map.save(f'{region_name}_heatmap.html')

    print(f'{region_name} heatmap created')


In [ ]:
regions = Trait['region_txt'].unique()
for region in regions:
    region_data = Trait[Trait['region_txt'] == region]
    create_heatmap_for_region(region_data, region)


In [ ]:
# Créer un dataframe pour comptabiliser le nombre d'événements par pays et continent
events_by_country = geo.groupby(['region_txt', 'country_txt']).size().reset_index(name='count')

# Boucle pour créer une carte pour chaque continent
for continent in events_by_country['region_txt'].unique():

    # Créer un dataframe pour le continent en cours
    continent_df = events_by_country[events_by_country['region_txt'] == continent]

   # Obtenir les informations de localisation pour chaque pays dans le continent
    country_locations = geo.loc[geo['region_txt'] == continent, ['latitude', 'longitude', 'country_txt']].drop_duplicates()

    # Créer une carte centrée sur la position moyenne des événements pour ce continent
    map = folium.Map(location=[country_locations['latitude'].mean(), country_locations['longitude'].mean()], zoom_start=4)

    # Ajouter un marqueur pour chaque pays dans le continent
    for index, row in country_locations.iterrows():
     folium.Marker(location=[row['latitude'], row['longitude']], popup=row['country_txt'] + ': ' + str(events_by_country[(events_by_country['region_txt'] == continent) & (events_by_country['country_txt'] == row['country_txt'])]['count'].values[0])).add_to(map)

    # Enregistrer la carte
    map.save(continent + '_events_map.html')
